In [3]:
import psycopg2

try:
    connection = psycopg2.connect(
        user="user",
        port="port",
        host="localhost",
        database="name"
    )
    
    cursor = connection.cursor()

    # The table sentiment_analysis_data_msft contains tens of thousands of news articles about MSFT  
    select_query = "SELECT * FROM sentiment_analysis_data_msft;"

    cursor.execute(select_query)

    # Fetch all records
    records_msft = cursor.fetchall()
    print(records_msft[700][3])
    
except Exception as error:
    print(f"Failed to fetch records: {error}")

finally:
    # Close the connection
    if connection:
        cursor.close()
        connection.close()
        print("Connection closed.")

Microsoft Corporation has recently announced that an extended line-up of gaming titles, technologies and partner support will accompany its premier gaming platform, Windows Vista. The move will enhance the gaming capabilities for PC owners when Windows Vista is launched in the United States on January 30. The announcement was made at the 2007 International Consumer Electronics Show in Las Vegas, the world's largest annual tradeshow for consumer technology. During the event, Microsoft Chairman Bill Gates demonstrated the networking capabilities of Windows Vista-based games with an Xbox 360 video game and entertainment system, highlighting the ability of gamers to play and communicate online across multiple gaming platforms. Commenting on the launch, Armagan Demir, Head of Entertainment and Devices Division Middle East, Microsoft said, "The countdown has begun for a new era of gameplay, as Windows Vista will usher in a revolutionary user experience that allows more players to connect in 

In [4]:
connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

drop_table_query = "DROP TABLE IF EXISTS sentiment_data_msft;"
cursor.execute(drop_table_query)
connection.commit()
print("Table dropped successfully.")

create_table_query = '''CREATE TABLE sentiment_data_msft (
    ID SERIAL PRIMARY KEY,
    Title TEXT,
    Time_Published TEXT,
    Sentiment_Score REAL
); '''

cursor.execute(create_table_query)
connection.commit()
print("Table Created Successfully")

Table dropped successfully.
Table Created Successfully


In [6]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from nltk.tokenize import sent_tokenize
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

# Load the model and tokenizer
model_name = "nickmuchi/deberta-v3-base-finetuned-finance-text-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)  # Moved to device
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocessing function
def preprocess(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return inputs

# Inference function for a single sentence
def get_sentiment(text):
    inputs = preprocess(text).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    label_id = torch.argmax(probabilities).item()
    return label_id

# Your articles
article_text = [record[3] for record in records_msft]
article_date = [record[2] for record in records_msft]
article_title = [record[1] for record in records_msft]

# Iterate over articles
for i, (article, date, title) in enumerate(zip(article_text, article_date, article_title)):
    sentences = sent_tokenize(article)  # Split the article into sentences
    sentiment_scores = [get_sentiment(sentence) for sentence in sentences]  # Get sentiment for each sentence
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)  # Average the sentiment scores
    cursor.execute("INSERT INTO sentiment_data_msft (ID, Title, Sentiment_Score, Time_Published) VALUES (%s, %s, %s, %s)",
                   (i, title, avg_sentiment, date))
    connection.commit()
    print(f"Average Sentiment of Article {i+1} is {avg_sentiment} at {date} for {title}")


Average Sentiment of Article 1 is 1.0 at April 27, 2007 Friday 9:47 AM EST for Briefing.com : Upgrades/Downgrades
Average Sentiment of Article 2 is 1.186046511627907 at April 23, 2007 Monday 2:56 PM GMT for HP, Microsoft and SAP Deliver Appliance for Duet(TM) Software
Average Sentiment of Article 3 is 1.0303030303030303 at April 3, 2007 Tuesday 12:41 AM EST for Free Online Courses at HP's Small Biz Website
Average Sentiment of Article 4 is 1.0 at April 17, 2007 Tuesday 7:07 AM EST for Microsoft expands Windows Genuine Advantage Notification
Average Sentiment of Article 5 is 0.9629629629629629 at April 2, 2007 Monday 16:36 PM EST for Monday's Tech Winners & Losers
Average Sentiment of Article 6 is 0.9130434782608695 at April 25, 2007 Wednesday 4:54 PM EST for Free Tool Recovers Deleted Files
Average Sentiment of Article 7 is 1.0 at April 5, 2007 Thursday 4:01 AM GMT for 'Is Your Family Set?' Campaign Rolls Into Michigan; Campaign aims to
Average Sentiment of Article 8 is 1.1964285714285